## Data input for BIDS datasets
`DataGrabber` and `SelectFiles` are great if you are dealing with generic datasets with arbitrary organization. However if you have decided to use Brain Imaging Data Structure (BIDS) to organized your data (or got your hands on a BIDS dataset) you can take advanted of a formal structure BIDS imposes. In this short tutorial you will learn how to do this.

## `pybids` - a Python API for working with BIDS datasets
`pybids` is a lightweight python API for querying BIDS folder structure for specific files and metadata. You can install it from PyPi:
```
pip install pybids
```
Please note it should be already installed in the tutorial Docker image.

## The `layout` object and simple queries
To begin working with pubids we need to initalize a layout object. We will need it to do all of our queries

In [1]:
from bids.grabbids import BIDSLayout
layout = BIDSLayout("/home/jovyan/work/data/ds000114/")

In [2]:
!tree -I derivatives /home/jovyan/work/data/ds000114/

/home/jovyan/work/data/ds000114/
├── dataset_description.json
├── dwi.bval
├── dwi.bvec
├── sub-01
│   ├── anat
│   │   └── sub-01_T1w.nii.gz -> ../../.git/annex/objects/QP/jm/MD5E-s8677710--d6820f6cb8fb965e864419c14f6a22d5.nii.gz/MD5E-s8677710--d6820f6cb8fb965e864419c14f6a22d5.nii.gz
│   ├── dwi
│   │   └── sub-01_dwi.nii.gz -> ../../.git/annex/objects/X2/kJ/MD5E-s104193417--454850ac9ed1c96c40b6b7946e51bf71.nii.gz/MD5E-s104193417--454850ac9ed1c96c40b6b7946e51bf71.nii.gz
│   └── func
│       ├── sub-01_task-fingerfootlips_bold.nii.gz -> ../../.git/annex/objects/k6/4f/MD5E-s24454931--e9ab535d84a922b0c7ed52461244cf47.nii.gz/MD5E-s24454931--e9ab535d84a922b0c7ed52461244cf47.nii.gz
│       ├── sub-01_task-linebisection_bold.nii.gz -> ../../.git/annex/objects/32/Qq/MD5E-s31617092--151bc230c3b577110883369b6fad0daa.nii.gz/MD5E-s31617092--151bc230c3b577110883369b6fad0daa.nii.gz
│       └── sub-01_task-linebisection_events.tsv
├── sub-02
│   ├── anat
│   │   └── sub-02_T1w.nii.gz -> ../../.git/a

Let's figure out what are the subject labels in this dataset

In [3]:
layout.get_subjects()

['01', '02']

What modalities are included in this dataset?

In [4]:
layout.get_modalities()

['anat', 'dwi', 'func']

What different data types are included in this dataset?

In [5]:
layout.get_types()

['bold', 'dwi', 'events', 'T1w']

In [6]:
layout.get_types(modality='func')

['bold', 'events']

What are the different tasks included in this dataset?

In [7]:
layout.get_tasks()

['fingerfootlips', 'linebisection']

We can also ask for all of the data for a particular subject.

In [8]:
layout.get(subject='01')

[File(filename='/home/jovyan/work/data/ds000114/derivatives/freesurfer/sub-01/label/aparc.annot.a2009s.ctab', subject='01'),
 File(filename='/home/jovyan/work/data/ds000114/derivatives/freesurfer/sub-01/label/aparc.annot.ctab', subject='01'),
 File(filename='/home/jovyan/work/data/ds000114/derivatives/freesurfer/sub-01/label/aparc.annot.DKTatlas40.ctab', subject='01'),
 File(filename='/home/jovyan/work/data/ds000114/derivatives/freesurfer/sub-01/label/BA.ctab', subject='01'),
 File(filename='/home/jovyan/work/data/ds000114/derivatives/freesurfer/sub-01/label/BA.thresh.ctab', subject='01'),
 File(filename='/home/jovyan/work/data/ds000114/derivatives/freesurfer/sub-01/label/lh.aparc.a2009s.annot', subject='01'),
 File(filename='/home/jovyan/work/data/ds000114/derivatives/freesurfer/sub-01/label/lh.aparc.annot', subject='01'),
 File(filename='/home/jovyan/work/data/ds000114/derivatives/freesurfer/sub-01/label/lh.aparc.DKTatlas40.annot', subject='01'),
 File(filename='/home/jovyan/work/dat

We can also ask for a specific subset of data. Note that we are using extension filter to get just the imaging data (BIDS allows both .nii and .nii.gz so we need to include both).

In [9]:
layout.get(subject='01', type='bold', extensions=['nii', 'nii.gz'])

[File(filename='/home/jovyan/work/data/ds000114/sub-01/func/sub-01_task-fingerfootlips_bold.nii.gz', task='fingerfootlips', subject='01', modality='func', type='bold'),
 File(filename='/home/jovyan/work/data/ds000114/sub-01/func/sub-01_task-linebisection_bold.nii.gz', task='linebisection', subject='01', modality='func', type='bold')]

You probably noticed that this method does not only return the file paths, but objects with relevant query fields. We can easily extract just the file paths.

In [10]:
[f.filename for f in layout.get(subject='01', type='T1w', extensions=['nii', 'nii.gz'])]

['/home/jovyan/work/data/ds000114/sub-01/anat/sub-01_T1w.nii.gz']

### Exercise 1:
List all of the BOLD files for subject 03, but only for the linebisection task

## Including `pybids` in your `nipype` workflow
This is great, but what we really want is to include this into our `nipype` workflows. How to do this? We can create our own custom `BIDSDataGrabber` using a `Function` Interface. First we need a plain Python function that for a given subject label and dataset location will return list of BOLD and T1w files.

In [11]:
def get_niftis(subject_id, data_dir):
    # Remember that all the necesary imports need to be INSIDE the function for the Function Interface to work!
    from bids.grabbids import BIDSLayout
    
    layout = BIDSLayout(data_dir)
    
    bolds = [f.filename for f in layout.get(subject=subject_id, type='bold', extensions=['nii', 'nii.gz'])]
    
    return bolds

In [12]:
get_niftis('01', '/home/jovyan/work/data/ds000114/')

['/home/jovyan/work/data/ds000114/sub-01/func/sub-01_task-fingerfootlips_bold.nii.gz',
 '/home/jovyan/work/data/ds000114/sub-01/func/sub-01_task-linebisection_bold.nii.gz']

Ok we got our function. Now we need to wrap it inside a Node object.

In [13]:
from nipype.pipeline import Node, MapNode, Workflow
from nipype.interfaces.utility import IdentityInterface, Function

In [14]:
BIDSDataGrabber = Node(Function(function=get_niftis, input_names=["subject_id",
                                       "data_dir"],
                                   output_names=["bolds", 
                                        "T1ws"]), name="BIDSDataGrabber")
BIDSDataGrabber.inputs.data_dir = "/home/jovyan/work/data/ds000114/"

In [15]:
BIDSDataGrabber.inputs.subject_id='01'
res = BIDSDataGrabber.run()
res.outputs

170325-01:31:11,243 workflow INFO:
	 Executing node BIDSDataGrabber in dir: /tmp/tmpfbaoru3k/BIDSDataGrabber



T1ws = /home/jovyan/work/data/ds000114/sub-01/func/sub-01_task-linebisection_bold.nii.gz
bolds = /home/jovyan/work/data/ds000114/sub-01/func/sub-01_task-fingerfootlips_bold.nii.gz

Works like a charm! (hopefully :) Lets put it in a workflow. We are not going to analyze any data, but for demostrantion purposes we will add a couple of nodes that pretend to analyze their inputs

In [16]:
def printMe(paths):
    print("\n\nanalyzing " + str(paths) + "\n\n")
    
analyzeBOLD = Node(Function(function=printMe, input_names=["paths"],
                            output_names=[]), name="analyzeBOLD")

In [17]:
wf = Workflow(name="bids_demo")
wf.connect(BIDSDataGrabber, "bolds", analyzeBOLD, "paths")
wf.run()

170325-01:31:12,752 workflow INFO:
	 Workflow bids_demo settings: ['check', 'execution', 'logging']
170325-01:31:12,900 workflow INFO:
	 Running serially.
170325-01:31:12,903 workflow INFO:
	 Executing node BIDSDataGrabber in dir: /tmp/tmpyml_3dg_/bids_demo/BIDSDataGrabber
170325-01:31:14,44 workflow INFO:
	 Executing node analyzeBOLD in dir: /tmp/tmpaf7epqfc/bids_demo/analyzeBOLD


analyzing /home/jovyan/work/data/ds000114/sub-01/func/sub-01_task-fingerfootlips_bold.nii.gz




### Exercise 2:
Modify the `BIDSDataGrabber` and the workflow to include T1ws.

## Iterating over subject labels
In the previous example we demostrated how to use `pybids` to "analyze" one subject. How can we scale it for all subjects? Easy - using `iterables`.

In [18]:
BIDSDataGrabber.iterables = ('subject_id', layout.get_subjects())
wf.run()

170325-01:31:15,283 workflow INFO:
	 Workflow bids_demo settings: ['check', 'execution', 'logging']
170325-01:31:15,419 workflow INFO:
	 Running serially.
170325-01:31:15,420 workflow INFO:
	 Executing node BIDSDataGrabber.aI.a1 in dir: /tmp/tmp1_1rge6u/bids_demo/_subject_id_02/BIDSDataGrabber
170325-01:31:16,771 workflow INFO:
	 Executing node BIDSDataGrabber.aI.a0 in dir: /tmp/tmppm8l31ei/bids_demo/_subject_id_01/BIDSDataGrabber
170325-01:31:18,117 workflow INFO:
	 Executing node analyzeBOLD.a0 in dir: /tmp/tmpoxqxt2c0/bids_demo/_subject_id_01/analyzeBOLD


analyzing /home/jovyan/work/data/ds000114/sub-01/func/sub-01_task-fingerfootlips_bold.nii.gz


170325-01:31:18,147 workflow INFO:
	 Executing node analyzeBOLD.a1 in dir: /tmp/tmp6sl3ajui/bids_demo/_subject_id_02/analyzeBOLD


analyzing /home/jovyan/work/data/ds000114/sub-02/func/sub-02_task-fingerfootlips_bold.nii.gz




## Accessing additional metadata
Querying different files is nice, but sometimes you want to access more metadata. For example `RepetitionTime`. `pybids` can help with that as well

In [19]:
layout.get_metadata('/home/jovyan/work/data/ds000114/sub-01/func/sub-01_task-fingerfootlips_bold.nii.gz')

{'EchoTime': 0.05,
 'FlipAngle': 90,
 'RepetitionTime': 2.5,
 'SliceTiming': [0.0,
  1.2499999999999998,
  0.08333333333333333,
  1.333333333333333,
  0.16666666666666666,
  1.4166666666666663,
  0.25,
  1.4999999999999996,
  0.3333333333333333,
  1.5833333333333328,
  0.41666666666666663,
  1.666666666666666,
  0.5,
  1.7499999999999993,
  0.5833333333333333,
  1.8333333333333326,
  0.6666666666666666,
  1.9166666666666659,
  0.75,
  1.9999999999999991,
  0.8333333333333333,
  2.083333333333332,
  0.9166666666666666,
  2.1666666666666656,
  1.0,
  2.249999999999999,
  1.0833333333333333,
  2.333333333333332,
  1.1666666666666665,
  2.416666666666665],
 'TaskName': 'finger_foot_lips'}

Can we incorporate this into our pipeline? Yes we can!

In [20]:
def printMetadata(path, data_dir):
    from bids.grabbids import BIDSLayout
    layout = BIDSLayout(data_dir)
    print("\n\nanalyzing " + path + "\nTR: "+ str(layout.get_metadata(path)["RepetitionTime"]) + "\n\n")
    
analyzeBOLD2 = MapNode(Function(function=printMetadata, input_names=["path", "data_dir"],
                             output_names=[]), name="analyzeBOLD2", iterfield="path")
analyzeBOLD2.inputs.data_dir = "/home/jovyan/work/data/ds000114/"

In [21]:
wf = Workflow(name="bids_demo")
wf.connect(BIDSDataGrabber, "bolds", analyzeBOLD2, "path")
wf.run()

170325-01:31:18,255 workflow INFO:
	 Workflow bids_demo settings: ['check', 'execution', 'logging']
170325-01:31:18,407 workflow INFO:
	 Running serially.
170325-01:31:18,409 workflow INFO:
	 Executing node BIDSDataGrabber.aI.a1 in dir: /tmp/tmpez6iduom/bids_demo/_subject_id_02/BIDSDataGrabber
170325-01:31:19,658 workflow INFO:
	 Executing node analyzeBOLD2.a1 in dir: /tmp/tmpl_vaqwqm/bids_demo/_subject_id_02/analyzeBOLD2
170325-01:31:19,689 workflow INFO:
	 Executing node _analyzeBOLD20 in dir: /tmp/tmpl_vaqwqm/bids_demo/_subject_id_02/analyzeBOLD2/mapflow/_analyzeBOLD20


analyzing /home/jovyan/work/data/ds000114/sub-02/func/sub-02_task-fingerfootlips_bold.nii.gz
TR: 2.5


170325-01:31:20,831 workflow INFO:
	 Executing node BIDSDataGrabber.aI.a0 in dir: /tmp/tmpf9nj9ys5/bids_demo/_subject_id_01/BIDSDataGrabber
170325-01:31:21,981 workflow INFO:
	 Executing node analyzeBOLD2.a0 in dir: /tmp/tmpiapxqcwq/bids_demo/_subject_id_01/analyzeBOLD2
170325-01:31:21,991 workflow INFO:
	 Executin

### Exercise 3:
Modify the `printMetadata` function to also print `EchoTime` 